In [3]:
import pandas as pd
import numpy as np
import json
import copy
# import h5py

In [10]:
# Read files
## Final demand matrix
f = open('/mnt/c/Users/Shiyuan Wang/Desktop/Global_PM2.5/EORA/EURO26/Eora26_2016_bp_FD.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")
    newData[i][-1] = newData[i][-1][:-1]
FD_df = pd.DataFrame(newData)
f.close()

f = open('/mnt/c/Users/Shiyuan Wang/Desktop/Global_PM2.5/EORA/EURO26/labels_T.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")[:-1]
LABELS_df = pd.DataFrame(newData)
f.close()

## Intermediate transaction matrix
f = open('/mnt/c/Users/Shiyuan Wang/Desktop/Global_PM2.5/EORA/EURO26/Eora26_2016_bp_T.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")
    newData[i][-1] = newData[i][-1][:-1]
T_df = pd.DataFrame(newData)
f.close()

In [7]:
# From str to float : T_df, FD_df
T_df = T_df.astype('float')
FD_df = FD_df.astype('float');

In [8]:
T_df = T_df.drop(columns=list(range(120*26,121*26)) + list(range(174*26,175*26)))
T_df = T_df.drop(list(range(120*26,121*26)) + list(range(174*26,175*26)))
FD_df = FD_df.drop(columns=list(range(120*6,121*6)) + list(range(174*6,175*6))) 
FD_df = FD_df.drop(list(range(120*26,121*26)) + list(range(174*26,175*26))) 
LABELS_df = LABELS_df.drop(list(range(120*26,121*26)) + list(range(174*26,175*26)))

In [9]:
T_df.columns = list(range(0,4863))
FD_df.columns = list(range(0,1128))

In [10]:
Countries = list(LABELS_df[0].unique())

In [11]:
T_df

,0,1,2,3,4,5,6,7,8,9,...,4853,4854,4855,4856,4857,4858,4859,4860,4861,4862
0,98135.00000,25.30470,359.37200,214505.00000,2674.45000,32478.10000,2813.46000,139.37000,186.41400,19.09510,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.430900e-07,1.000000e-07,1.000000e-07,398916.00
1,132.40800,418.24000,2.32455,28665.90000,4.19821,1.56355,35.82680,18.74070,2.47318,1.07425,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,28619.20
2,1603.91000,37.79110,10299.00000,667.63800,68.57620,698.94000,62054.80000,6174.97000,188.68600,265.54800,...,1.048770e-07,1.043120e-07,1.042980e-07,1.000000e-07,1.001720e-07,1.002060e-07,1.000000e-07,1.000000e-07,1.000000e-07,37647.30
3,40408.80000,468.59600,54.37240,161090.00000,1532.57000,1237.13000,4558.57000,28.90200,54.17510,13.63440,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,5095.01
4,1706.29000,219.67600,385.27000,1929.72000,50932.00000,4957.07000,6176.40000,919.84400,4939.74000,3683.98000,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,20594.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,3.01483,3.20941,4.24502,5.19153,5.24824,5.24130,5.16952,5.18477,5.10443,5.17863,...,1.097940e+00,7.491060e-01,7.833540e-01,3.013900e-01,4.209160e-01,1.014600e-01,3.243420e+01,2.166330e+01,2.714370e+01,367530.00
4911,1.17719,1.37551,1.84375,2.25750,2.27674,2.29281,2.26491,2.26818,2.24488,2.25501,...,9.593950e-02,1.035910e-01,9.461920e-02,1.078180e-01,9.560480e-02,9.890780e-02,1.014600e-01,5.840000e-01,4.603710e-01,40930.00
4912,3.30335,3.58883,4.79122,5.92975,5.93691,5.97308,5.90872,5.91886,5.85291,5.88518,...,7.730420e-02,8.454030e-02,7.691460e-02,8.448920e-02,7.893080e-02,7.932750e-02,7.433410e-01,1.014600e-01,5.172770e-01,145728.00
4913,5.94325,6.83916,8.99569,11.27200,11.27610,11.35070,11.24740,11.25540,11.12770,11.17790,...,8.463560e-02,9.186710e-02,8.350150e-02,9.163710e-02,8.236980e-02,8.578660e-02,1.438470e+00,5.986140e-01,1.014600e-01,179240.00


In [12]:
FD_df

,0,1,2,3,4,5,6,7,8,9,...,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127
0,2.441240e+06,21191.40000,4170.41000,17477.90000,9921.32000,50.29780,1.369080,1.338850,2.235760,2.504540,...,1.067240,1.420560,0.030502,1.050150,0.0,0.0,0.0,0.0,0.0,0.0
1,9.756520e+04,526.74900,2603.48000,7.44061,793.11800,7.44061,0.210492,0.210492,0.244429,0.255776,...,0.709638,0.598005,0.020051,0.598005,0.0,0.0,0.0,0.0,0.0,0.0
2,1.923700e+04,18.99120,334.61100,548.32100,1452.70000,1.78847,0.156236,0.156236,0.246782,0.215100,...,0.571534,0.398656,0.020445,0.398656,0.0,0.0,0.0,0.0,0.0,0.0
3,8.358320e+05,11606.10000,1.04009,0.44913,1812.68000,0.44913,0.056894,0.056894,0.080191,0.070835,...,0.398171,0.280365,0.015234,0.280365,0.0,0.0,0.0,0.0,0.0,0.0
4,1.393990e+05,1849.85000,28.03080,5638.92000,329.13900,13.14740,0.097453,0.097453,0.173592,0.149557,...,0.425890,0.348862,0.022607,0.348862,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,6.406050e+00,6.40605,7.61509,7.99647,3.62230,7.99647,5.176600,5.176600,5.871570,6.588000,...,301040.000000,71759.100000,0.000000,71759.100000,0.0,0.0,0.0,0.0,0.0,0.0
4911,2.969850e+00,2.96985,3.40420,3.24971,1.71085,3.24971,1.817520,1.817520,1.933540,2.008380,...,109792.000000,21763.100000,0.000000,21763.100000,0.0,0.0,0.0,0.0,0.0,0.0
4912,7.513970e+00,7.51397,8.74698,8.27024,3.83962,8.27024,5.790060,5.790060,6.324450,6.546780,...,117676.000000,22373.000000,0.000000,22373.000000,0.0,0.0,0.0,0.0,0.0,0.0
4913,1.449490e+01,14.49490,17.07700,15.95110,6.92056,15.95110,11.069300,11.069300,12.226400,12.927700,...,134683.000000,25011.100000,0.000000,25011.100000,0.0,0.0,0.0,0.0,0.0,0.0


## demand by countries
Each contry's final demand from each sector.

In [13]:
size = len(FD_df)

In [14]:
FD_df

,0,1,2,3,4,5,6,7,8,9,...,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127
0,2.441240e+06,21191.40000,4170.41000,17477.90000,9921.32000,50.29780,1.369080,1.338850,2.235760,2.504540,...,1.067240,1.420560,0.030502,1.050150,0.0,0.0,0.0,0.0,0.0,0.0
1,9.756520e+04,526.74900,2603.48000,7.44061,793.11800,7.44061,0.210492,0.210492,0.244429,0.255776,...,0.709638,0.598005,0.020051,0.598005,0.0,0.0,0.0,0.0,0.0,0.0
2,1.923700e+04,18.99120,334.61100,548.32100,1452.70000,1.78847,0.156236,0.156236,0.246782,0.215100,...,0.571534,0.398656,0.020445,0.398656,0.0,0.0,0.0,0.0,0.0,0.0
3,8.358320e+05,11606.10000,1.04009,0.44913,1812.68000,0.44913,0.056894,0.056894,0.080191,0.070835,...,0.398171,0.280365,0.015234,0.280365,0.0,0.0,0.0,0.0,0.0,0.0
4,1.393990e+05,1849.85000,28.03080,5638.92000,329.13900,13.14740,0.097453,0.097453,0.173592,0.149557,...,0.425890,0.348862,0.022607,0.348862,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,6.406050e+00,6.40605,7.61509,7.99647,3.62230,7.99647,5.176600,5.176600,5.871570,6.588000,...,301040.000000,71759.100000,0.000000,71759.100000,0.0,0.0,0.0,0.0,0.0,0.0
4911,2.969850e+00,2.96985,3.40420,3.24971,1.71085,3.24971,1.817520,1.817520,1.933540,2.008380,...,109792.000000,21763.100000,0.000000,21763.100000,0.0,0.0,0.0,0.0,0.0,0.0
4912,7.513970e+00,7.51397,8.74698,8.27024,3.83962,8.27024,5.790060,5.790060,6.324450,6.546780,...,117676.000000,22373.000000,0.000000,22373.000000,0.0,0.0,0.0,0.0,0.0,0.0
4913,1.449490e+01,14.49490,17.07700,15.95110,6.92056,15.95110,11.069300,11.069300,12.226400,12.927700,...,134683.000000,25011.100000,0.000000,25011.100000,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
2.441240e+06+21191.40000+4170.41000+17477.90000+9921.32000+50.29780

2494051.3277999996

In [16]:
for i in range(len(Countries)):
    for j in range(10):
        print('haha',Countries[i],'j',j,)

haha AFG j 0
haha AFG j 1
haha AFG j 2
haha AFG j 3
haha AFG j 4
haha AFG j 5
haha AFG j 6
haha AFG j 7
haha AFG j 8
haha AFG j 9
haha ALB j 0
haha ALB j 1
haha ALB j 2
haha ALB j 3
haha ALB j 4
haha ALB j 5
haha ALB j 6
haha ALB j 7
haha ALB j 8
haha ALB j 9
haha DZA j 0
haha DZA j 1
haha DZA j 2
haha DZA j 3
haha DZA j 4
haha DZA j 5
haha DZA j 6
haha DZA j 7
haha DZA j 8
haha DZA j 9
haha AND j 0
haha AND j 1
haha AND j 2
haha AND j 3
haha AND j 4
haha AND j 5
haha AND j 6
haha AND j 7
haha AND j 8
haha AND j 9
haha AGO j 0
haha AGO j 1
haha AGO j 2
haha AGO j 3
haha AGO j 4
haha AGO j 5
haha AGO j 6
haha AGO j 7
haha AGO j 8
haha AGO j 9
haha ATG j 0
haha ATG j 1
haha ATG j 2
haha ATG j 3
haha ATG j 4
haha ATG j 5
haha ATG j 6
haha ATG j 7
haha ATG j 8
haha ATG j 9
haha ARG j 0
haha ARG j 1
haha ARG j 2
haha ARG j 3
haha ARG j 4
haha ARG j 5
haha ARG j 6
haha ARG j 7
haha ARG j 8
haha ARG j 9
haha ARM j 0
haha ARM j 1
haha ARM j 2
haha ARM j 3
haha ARM j 4
haha ARM j 5
haha ARM j 6

haha DEU j 8
haha DEU j 9
haha GHA j 0
haha GHA j 1
haha GHA j 2
haha GHA j 3
haha GHA j 4
haha GHA j 5
haha GHA j 6
haha GHA j 7
haha GHA j 8
haha GHA j 9
haha GRC j 0
haha GRC j 1
haha GRC j 2
haha GRC j 3
haha GRC j 4
haha GRC j 5
haha GRC j 6
haha GRC j 7
haha GRC j 8
haha GRC j 9
haha GRL j 0
haha GRL j 1
haha GRL j 2
haha GRL j 3
haha GRL j 4
haha GRL j 5
haha GRL j 6
haha GRL j 7
haha GRL j 8
haha GRL j 9
haha GTM j 0
haha GTM j 1
haha GTM j 2
haha GTM j 3
haha GTM j 4
haha GTM j 5
haha GTM j 6
haha GTM j 7
haha GTM j 8
haha GTM j 9
haha GIN j 0
haha GIN j 1
haha GIN j 2
haha GIN j 3
haha GIN j 4
haha GIN j 5
haha GIN j 6
haha GIN j 7
haha GIN j 8
haha GIN j 9
haha GUY j 0
haha GUY j 1
haha GUY j 2
haha GUY j 3
haha GUY j 4
haha GUY j 5
haha GUY j 6
haha GUY j 7
haha GUY j 8
haha GUY j 9
haha HTI j 0
haha HTI j 1
haha HTI j 2
haha HTI j 3
haha HTI j 4
haha HTI j 5
haha HTI j 6
haha HTI j 7
haha HTI j 8
haha HTI j 9
haha HND j 0
haha HND j 1
haha HND j 2
haha HND j 3
haha HND j 4

haha QAT j 8
haha QAT j 9
haha KOR j 0
haha KOR j 1
haha KOR j 2
haha KOR j 3
haha KOR j 4
haha KOR j 5
haha KOR j 6
haha KOR j 7
haha KOR j 8
haha KOR j 9
haha MDA j 0
haha MDA j 1
haha MDA j 2
haha MDA j 3
haha MDA j 4
haha MDA j 5
haha MDA j 6
haha MDA j 7
haha MDA j 8
haha MDA j 9
haha ROU j 0
haha ROU j 1
haha ROU j 2
haha ROU j 3
haha ROU j 4
haha ROU j 5
haha ROU j 6
haha ROU j 7
haha ROU j 8
haha ROU j 9
haha RUS j 0
haha RUS j 1
haha RUS j 2
haha RUS j 3
haha RUS j 4
haha RUS j 5
haha RUS j 6
haha RUS j 7
haha RUS j 8
haha RUS j 9
haha RWA j 0
haha RWA j 1
haha RWA j 2
haha RWA j 3
haha RWA j 4
haha RWA j 5
haha RWA j 6
haha RWA j 7
haha RWA j 8
haha RWA j 9
haha WSM j 0
haha WSM j 1
haha WSM j 2
haha WSM j 3
haha WSM j 4
haha WSM j 5
haha WSM j 6
haha WSM j 7
haha WSM j 8
haha WSM j 9
haha SMR j 0
haha SMR j 1
haha SMR j 2
haha SMR j 3
haha SMR j 4
haha SMR j 5
haha SMR j 6
haha SMR j 7
haha SMR j 8
haha SMR j 9
haha STP j 0
haha STP j 1
haha STP j 2
haha STP j 3
haha STP j 4

In [17]:
demand_by_countries = {}
for i in range(len(Countries)):
    demand_by_countries[Countries[i]] = np.zeros(size).reshape(-1, 1)

In [18]:
for i in range(len(Countries)):
    for j in range(size):
        demand_by_countries[Countries[i]][j] = sum(FD_df.iloc[j,6*i:6*(i+1)])

In [19]:
count = 0
for j in range(len(Countries)):
    print(26*j,26*(j+1),len(demand_by_countries['CHN'][26*j:26*(j+1)]),sum(demand_by_countries['CHN'][26*j:26*(j+1)])[0])
    count += 1
count

0 26 26 5672.064501000001
26 52 26 2127.60421
52 78 26 3212.0059848999995
78 104 26 1622.6368230000003
104 130 26 713798.8541303999
130 156 26 1116.2940943
156 182 26 863801.8431110049
182 208 26 3137.829267
208 234 26 718.348361
234 260 26 12169713.614452003
260 286 26 6581973.663856002
286 312 26 6751.354345
312 338 26 1632.329598
338 364 26 161401.02803900003
364 390 26 978426.539554
390 416 26 1322.843892
416 442 26 1619.35696
442 468 26 2994717.3764429996
468 494 26 824.019301
494 520 26 22357.367209000007
520 546 26 1665.5853806000002
546 572 26 472.8812724
572 598 26 3216.8762783000006
598 624 26 4259.583162999999
624 650 26 3223.9520800000005
650 676 26 3746888.5521662
676 702 26 1055.143307
702 728 26 11214.078672499998
728 754 26 198572.38315500002
754 780 26 1168.191482
780 806 26 387.7068488000001
806 832 26 249590.38828999997
832 858 26 82916.53931230001
858 884 26 9750575.094495866
884 910 26 1019.6184382000001
910 936 26 3049.555557
936 962 26 558.7356061999999
962 988 2

188

In [20]:
demand_by_countries_2 = {}
for i in Countries:
    demand_by_countries_2[i] = {}

In [21]:
for i in Countries:
    for j in range(len(Countries)):
        demand_by_countries_2[i][Countries[j]] = sum(demand_by_countries[i][26*j:26*(j+1)])[0]

In [22]:
file=open("demand_by_countries.json",'w')
json.dump(demand_by_countries_2,file)
file.close()

In [23]:
# Summation of the value insde a dictionary
def dictsum(myDict):      
    sum = 0
    for i in myDict: 
        sum = sum + myDict[i]     
    return sum

In [24]:
final_demand = {}
for i in Countries:
    final_demand[i] = dictsum(demand_by_countries_2[i])

In [25]:
file=open("final_demand.json",'w')
json.dump(final_demand,file)
file.close()

In [26]:
# with open("demand_by_countries.json",'r') as load_f:
#     demand_by_countries = json.load(load_f)

In [27]:
# demand_by_countries

## Leontif Inverse

In [4]:
Export = pd.read_csv("../1_Country_Emission/Export_woANTUSR.csv")

In [5]:
T_df2 = copy.copy(T_df)
Transaction_matrix = T_df2.to_numpy()
gross_output = list(Export["Gross_output"])

NameError: name 'T_df' is not defined

In [30]:
# A
A = np.zeros(shape=(4863,4863))
for i in range(4863):
    if gross_output[i] == 0:
        A[:, i] = 0
        continue
    A[:, i] = Transaction_matrix[:, i] / gross_output[i] 

In [31]:
# (I-A)^-1
Leontif_Inverse = np.linalg.inv(np.matrix(np.identity(4863) - A))

In [32]:
Leontif_Inverse.shape

(4863, 4863)

## Output by sectors
$$(I-A)^{-1}×D=X$$

In [33]:
output_by_sectors = {}
for i in range(len(Countries)):
    output_by_sectors[Countries[i]] = np.zeros(size).reshape(-1, 1)

In [34]:
np.mat(demand_by_countries[Countries[i]])

matrix([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [35]:
for i in range(len(Countries)):
    output_by_sectors[Countries[i]] = Leontif_Inverse * np.mat(demand_by_countries[Countries[i]])

In [36]:
output_by_sectors['CHN']

matrix([[1.11532025e+05],
        [6.71257611e+03],
        [1.34401322e+04],
        ...,
        [3.20934607e+04],
        [4.09645226e+04],
        [8.78289584e+07]])

In [37]:
output_df = {}
for i in output_by_sectors:
    output_df[i] = list(np.array(output_by_sectors[i]).flatten())

In [38]:
output_df = pd.DataFrame(output_df)

In [39]:
output_df

,AFG,ALB,DZA,AND,AGO,ATG,ARG,ARM,ABW,AUS,...,USA,URY,UZB,VUT,VEN,VNM,YEM,ZMB,ZWE,ROW
0,2.856308e+06,176.213409,5.983903e+03,57.153897,2.586983e+03,23.736257,5.970265e+03,158.301079,41.873740,5.469302e+03,...,5.722012e+04,204.690406,1.722565e+03,41.077625,3.505718e+04,1325.294955,737.141226,295.540636,136.361578,0.0
1,1.475613e+05,11.276593,3.808465e+02,7.110071,1.857298e+02,5.733368,4.284619e+02,11.104175,5.621235,4.011548e+02,...,3.037890e+03,24.471248,1.424229e+02,7.198934,2.561749e+03,80.830009,43.839837,20.328259,12.621085,0.0
2,2.091753e+05,17.936255,7.604518e+02,9.891620,3.636822e+02,2.828431,8.344248e+02,12.362850,4.073849,6.849307e+02,...,5.722302e+03,23.549467,2.199963e+02,5.167928,5.105717e+03,135.230919,77.136788,30.462429,16.101443,0.0
3,1.277903e+06,16.232493,4.642511e+02,6.638122,2.186884e+02,1.620191,5.031463e+02,7.647916,2.671378,4.192627e+02,...,3.610613e+03,13.867495,1.309160e+02,2.884308,3.067060e+03,88.561540,48.014635,19.651501,11.253665,0.0
4,2.604547e+05,23.493306,4.149404e+02,16.865873,2.003794e+02,1.824174,4.539396e+02,7.713040,3.482697,3.778777e+02,...,3.264468e+03,29.429892,1.225078e+02,3.390701,2.763109e+03,80.101329,43.692512,19.173735,11.604357,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4858,2.269063e+02,182.888649,4.436876e+03,110.226823,2.112063e+03,127.452150,4.983938e+03,132.741967,96.509775,4.360630e+03,...,3.429980e+04,358.772185,1.679967e+03,128.570544,2.965561e+04,984.476925,527.664213,201.046516,905932.286432,0.0
4859,7.914737e+01,54.871977,5.255117e+02,42.297914,2.603470e+02,45.072288,6.256715e+02,40.726586,33.136637,6.436060e+02,...,4.555467e+03,108.764513,3.451307e+02,46.688555,3.424250e+03,229.053761,94.917148,46.571094,381539.951810,0.0
4860,2.151702e+02,178.983351,1.851462e+03,87.911193,9.124343e+02,136.842643,2.184557e+03,132.102423,103.870516,2.197608e+03,...,1.571313e+04,349.486691,1.144787e+03,141.582915,1.212582e+04,599.457668,302.875114,149.981643,377819.181544,0.0
4861,3.962601e+02,339.255818,2.380367e+03,143.003474,1.209281e+03,261.416380,2.916634e+03,250.838623,200.292536,3.217327e+03,...,2.177529e+04,665.086041,1.955036e+03,272.073651,1.533378e+04,928.925452,468.649334,266.373261,425784.622271,0.0


In [40]:
output_df.to_csv('output_matrix_eorasector.csv')

In [41]:
(len(output_by_sectors['CHN'])-1)/187

26.0

In [42]:
output_by_sectors['AFG'].shape

(4863, 1)

In [ ]:
output_df

In [7]:
output_df = pd.read_csv('output_matrix_eorasector.csv',index_col=0)

In [24]:
output_df['SAU']

0       7.664381e+03
1       1.376978e+02
2       2.561122e+02
3       2.129467e+02
4       2.641675e+02
            ...     
4858    1.510155e+03
4859    2.246737e+02
4860    7.639555e+02
4861    1.140377e+03
4862    1.511089e+06
Name: SAU, Length: 4863, dtype: float64

In [9]:
output_df.columns.get_loc("SAU")

145

In [14]:
LABELS_df[LABELS_df[3] == 'Mining and Quarrying']

,0,1,2,3
2,AFG,AFG,Industries,Mining and Quarrying
28,ALB,ALB,Industries,Mining and Quarrying
54,DZA,DZA,Industries,Mining and Quarrying
80,AND,AND,Industries,Mining and Quarrying
106,AGO,AGO,Industries,Mining and Quarrying
...,...,...,...,...
4786,VEN,VEN,Industries,Mining and Quarrying
4812,VNM,VNM,Commodities,Mining and Quarrying
4838,YEM,YEM,Industries,Mining and Quarrying
4864,ZMB,ZMB,Industries,Mining and Quarrying


In [20]:
output_df[(26*145+2):(26*145+3)].sum().sum() # total output of petroleum sector of SAU

128151561.4497699

In [9]:
128151561.4497699/452445089.9237076

0.283242241553288

In [21]:
output_df[26*145:26*(145+1)].sum().sum() # total output of SAU

1117462175.119343

In [25]:
output_df[26*145:26*(145+1)]

,AFG,ALB,DZA,AND,AGO,ATG,ARG,ARM,ABW,AUS,...,USA,URY,UZB,VUT,VEN,VNM,YEM,ZMB,ZWE,ROW
3770,374.826389,402.287349,9929.823137,90.714020,3779.393775,27.240180,8787.652994,166.870089,52.014128,21101.867148,...,1.488512e+05,313.764090,2289.122112,40.488345,50469.833702,3404.304359,1207.947343,649.089788,222.076419,0.0
3771,68.008441,41.088526,434.656379,17.454471,207.707910,11.858069,459.742605,34.913614,14.704111,1412.244038,...,7.733630e+03,71.118955,317.068142,19.659295,2382.127604,272.744562,92.121576,48.595601,24.684127,0.0
3772,19528.401677,49276.463396,370030.189664,4363.690207,130548.576268,1300.671319,323642.278030,11264.812055,4349.282415,422762.170614,...,3.417855e+07,32290.806988,42460.843272,515.514669,765541.282062,348434.792875,33070.529501,14356.677692,14186.636770,0.0
3773,279.432963,147.409219,6159.746446,49.084659,1792.411271,15.270320,4086.842154,93.313145,31.221164,24731.385581,...,1.085464e+05,169.829117,1118.716092,23.269576,22348.033722,2822.544322,845.103682,386.584605,184.631576,0.0
3774,213.130826,376.414557,7371.110309,34.927139,1129.042092,10.855250,2249.997862,74.770031,23.031874,7234.995883,...,1.715900e+05,133.459894,606.098880,16.061973,11058.545604,1514.391957,357.919170,438.707793,371.466812,0.0
3775,487.553291,356.204147,19029.738341,108.393084,4834.390307,24.212537,11673.227364,191.914395,53.217702,39039.895266,...,2.344002e+05,419.945580,2591.576416,31.600678,62639.695409,6586.359189,1547.621561,596.976896,326.354272,0.0
3776,6824.354896,2191.047145,72956.216647,390.501059,17563.467705,131.735874,59051.739604,1420.395136,343.765201,599747.157379,...,2.485145e+06,3852.388386,9165.175068,170.167051,166970.050897,114693.151397,10269.919170,5158.121293,3647.684133,0.0
3777,678.402315,747.828968,21564.637246,132.811343,5822.840080,31.242657,13970.273993,296.552589,71.293340,33931.339832,...,3.768721e+05,640.196775,3036.087186,35.377229,71776.680803,8395.817149,2190.227543,741.048873,387.987657,0.0
3778,972.947126,1045.059737,48561.654203,263.076801,13166.324701,47.340332,31196.783436,421.560440,117.084740,55982.987676,...,6.627034e+05,1019.742067,6588.987868,45.160849,172804.546517,11853.271300,3364.163695,1302.898465,685.459527,0.0
3779,260.432204,278.053430,9644.765344,87.425016,4222.194641,14.860992,9954.868427,114.695892,32.880564,15185.701680,...,1.474822e+05,277.588759,2222.346457,18.145067,57706.243737,2967.148791,918.632064,373.594133,185.182788,0.0


In [23]:
128151561.4497699/1117462175.119343

0.11468089417530712

## Output matrix

In [82]:
hahaha = {}

for j in Countries:
    hahaha[j] = {}
    
for j in Countries:
    for i in range(188):
        hahaha[j][Countries[i]] = output_by_sectors[j][i*26:(i+1)*26] 

In [83]:
hahaha['CHN']['AUS']

matrix([[ 5309065.58298223],
        [  147099.15297829],
        [23729469.09333535],
        [ 3726920.49693402],
        [ 1223227.08397828],
        [ 2760409.40638721],
        [ 7273081.71547924],
        [16883146.34406954],
        [13265590.27866899],
        [ 1073523.75054458],
        [  349889.15165025],
        [   47415.3730614 ],
        [ 1783566.84620898],
        [ 1286769.36504543],
        [  139629.30923589],
        [ 4615256.56050094],
        [ 2213494.66288711],
        [ 1281453.33631067],
        [ 5073650.56136945],
        [ 1981468.99676062],
        [21914634.97842598],
        [  374653.27038188],
        [ 2877286.70720076],
        [   26829.7369563 ],
        [  128292.78529833],
        [   79765.82795051]])

In [84]:
# Assign sectors in EORA to sectors in CEDS
sectors = ['AGR/RCOO','RCOO','IND','IND','IND','IND','IND','IND','IND','IND','IND','IND','ENE',
           'IND','RCOC','RCOC','RCOC','RCOC','NRTR/SHP','0','0','RCOC/RCOO','RCOC','RCOR','0','0']
agr = [0]

ene = [i for i, x in enumerate(sectors) if x == 'ENE']

ind = [i for i, x in enumerate(sectors) if x == 'IND']

nrtr = [sectors.index('NRTR/SHP')]

rcor = [i for i, x in enumerate(sectors) if x == 'RCOR']

rcoc = [i for i, x in enumerate(sectors) if x == 'RCOC']
rcoc.append(sectors.index('RCOC/RCOO'))

rcoo = [i for i, x in enumerate(sectors) if x == 'RCOO']
rcoo.append(sectors.index('RCOC/RCOO'))
rcoo.append(sectors.index('AGR/RCOO'))

shp = [sectors.index('NRTR/SHP')]

#  What belongs to IND or RCO also belongs to SLV
slv = list(np.unique(ind + rcor + rcoc + rcoo))

# Every sector will have waste.
wst = list(np.unique(slv + shp + nrtr + ene + agr))

A = ['agr','ene','ind','nrtr','rcor','rcoc','rcoo','shp','slv','wst']

sector_pos = {}
sector_pos['agr'] = [0]
sector_pos['ene'] = [i for i, x in enumerate(sectors) if x == 'ENE']

sector_pos['ind'] = [i for i, x in enumerate(sectors) if x == 'IND']

sector_pos['nrtr'] = [sectors.index('NRTR/SHP')]

sector_pos['rcor'] = [i for i, x in enumerate(sectors) if x == 'RCOR']

sector_pos['rcoc'] = [i for i, x in enumerate(sectors) if x == 'RCOC']
sector_pos['rcoc'].append(sectors.index('RCOC/RCOO'))

sector_pos['rcoo'] = [i for i, x in enumerate(sectors) if x == 'RCOO']
sector_pos['rcoo'].append(sectors.index('RCOC/RCOO'))
sector_pos['rcoo'].append(sectors.index('AGR/RCOO'))

sector_pos['shp'] = [sectors.index('NRTR/SHP')]

#  What belongs to IND or RCO also belongs to SLV
sector_pos['slv'] = list(np.unique(ind + rcor + rcoc + rcoo))

# Every sector will have waste.
sector_pos['wst'] = list(np.unique(slv + shp + nrtr + ene + agr))

In [85]:
final = {}
for i in Countries:
    final[i] = {}
    for j in Countries:
        if j != "ROW": 
            final[i][j] = {}
            for k in A:
                final[i][j][k] = {}   
        else:
            final[i][j] = hahaha[i][j]

In [86]:
for i in Countries:
    for j in Countries[:-1]:
        for k in A:
            final[i][j][k] = float(sum([hahaha[i][j][keys] for keys in eval(k)]))

In [33]:
final_new = {}
for i in Countries:
    final_new[i] = []
    for j in Countries[:-1]:
        for k in A:
            final_new[i].append(final[i][j][k])
    final_new[i].append(final[i]['ROW'].item())

In [34]:
mmm2 = pd.DataFrame(final_new)

In [35]:
mmm2

,AFG,ALB,DZA,AND,AGO,ATG,ARG,ARM,ABW,AUS,...,USA,URY,UZB,VUT,VEN,VNM,YEM,ZMB,ZWE,ROW
0,2.856308e+06,176.213409,5.983903e+03,57.153897,2.586983e+03,23.736257,5.970265e+03,158.301079,41.873740,5.469302e+03,...,5.722012e+04,204.690406,1.722565e+03,41.077625,3.505718e+04,1325.294955,737.141226,295.540636,1.363616e+02,0.0
1,6.144598e+05,14.733583,1.109047e+03,10.799468,5.200492e+02,1.740918,1.215362e+03,11.241857,2.978175,9.859652e+02,...,8.253731e+03,25.386376,2.785704e+02,2.386820,7.493586e+03,164.535507,103.085671,34.930123,1.766381e+01,0.0
2,1.061175e+07,329.169884,1.518304e+04,178.295860,7.167120e+03,31.809328,1.663092e+04,182.431224,57.280620,1.352184e+04,...,1.135906e+05,395.273385,3.972909e+03,52.899612,1.022894e+05,2416.237604,1455.174871,527.700091,2.535464e+02,0.0
3,2.545157e+06,75.950192,3.709997e+03,38.705319,1.745342e+03,7.648099,4.055800e+03,46.308077,14.932629,3.330087e+03,...,2.817432e+04,93.934973,9.701794e+02,12.423030,2.488895e+04,609.010629,357.794524,127.787396,5.599617e+01,0.0
4,6.317936e+04,17.735237,8.851561e+01,3.559681,4.641842e+01,1.213549,9.684820e+01,3.766226,2.305124,8.999861e+01,...,7.148223e+02,5.244698,3.829567e+01,2.392206,5.694808e+02,34.966239,12.871314,8.152645,6.438740e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866,5.133166e+02,384.895144,7.346020e+03,252.503048,3.522957e+03,282.569746,8.346261e+03,282.214257,212.151144,7.534442e+03,...,5.786571e+04,753.541626,3.143446e+03,288.675867,4.889999e+04,1832.031818,950.739815,394.193981,1.802986e+06,0.0
1867,1.370776e+02,102.424637,2.254750e+03,69.766654,1.075899e+03,72.364154,2.549181e+03,74.444746,54.164354,2.288175e+03,...,1.809806e+04,199.592210,8.916533e+02,74.150909,1.503510e+04,591.901356,278.655945,112.053317,5.753469e+05,0.0
1868,1.931054e+03,1420.996569,2.982484e+04,1019.787832,1.426301e+04,1026.520885,3.374774e+04,1043.562458,772.674959,3.017783e+04,...,2.345420e+05,2795.534148,1.214879e+04,1052.875766,1.988760e+05,7442.279644,3736.488200,1533.728262,8.950014e+06,0.0
1869,2.222956e+03,1636.605709,3.471759e+04,1169.855766,1.659674e+04,1178.951616,3.926862e+04,1201.681742,887.561982,3.508485e+04,...,2.730942e+05,3218.518276,1.406016e+04,1209.554269,2.315408e+05,8627.453161,4332.951420,1774.768085,1.009709e+07,0.0


## Get rid of ANT and USR

## Update: instead of getting rid of these two countries, using concentrations factors for the rest of the world; total output stays the same

In [36]:
# mmm2 = mmm2.drop(['ANT','USR'],axis=1)
# mmm2 = mmm2.drop(index=range(120*10,121*10),axis=0)
# mmm2 = mmm2.drop(index=range(174*10,175*10),axis=0)
# mmm2 = mmm2.reset_index(drop=True)

In [ ]:
mmm2 = pd.DataFrame(final_new)
mmm2.to_csv('output_matrix.csv')

## Check for correctness
### The total output from each sector.

In [ ]:
mmm2 = pd.read_csv('output_matrix.csv',index_col=0)

In [6]:
mmm2

,AFG,ALB,DZA,AND,AGO,ATG,ARG,ARM,ABW,AUS,...,USA,URY,UZB,VUT,VEN,VNM,YEM,ZMB,ZWE,ROW
0,2.856308e+06,176.213409,5.983903e+03,57.153897,2.586983e+03,23.736257,5.970265e+03,158.301079,41.873740,5.469302e+03,...,5.722012e+04,204.690406,1.722565e+03,41.077625,3.505718e+04,1325.294955,737.141226,295.540636,1.363616e+02,0.0
1,6.144598e+05,14.733583,1.109047e+03,10.799468,5.200492e+02,1.740918,1.215362e+03,11.241857,2.978175,9.859652e+02,...,8.253731e+03,25.386376,2.785704e+02,2.386820,7.493586e+03,164.535507,103.085671,34.930123,1.766381e+01,0.0
2,1.061175e+07,329.169884,1.518304e+04,178.295860,7.167120e+03,31.809328,1.663092e+04,182.431224,57.280620,1.352184e+04,...,1.135906e+05,395.273385,3.972909e+03,52.899612,1.022894e+05,2416.237604,1455.174871,527.700091,2.535464e+02,0.0
3,2.545157e+06,75.950192,3.709997e+03,38.705319,1.745342e+03,7.648099,4.055800e+03,46.308077,14.932629,3.330087e+03,...,2.817432e+04,93.934973,9.701794e+02,12.423030,2.488895e+04,609.010629,357.794524,127.787396,5.599617e+01,0.0
4,6.317936e+04,17.735237,8.851561e+01,3.559681,4.641842e+01,1.213549,9.684820e+01,3.766226,2.305124,8.999861e+01,...,7.148223e+02,5.244698,3.829567e+01,2.392206,5.694808e+02,34.966239,12.871314,8.152645,6.438740e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866,5.133166e+02,384.895144,7.346020e+03,252.503048,3.522957e+03,282.569746,8.346261e+03,282.214257,212.151144,7.534442e+03,...,5.786571e+04,753.541626,3.143446e+03,288.675867,4.889999e+04,1832.031818,950.739815,394.193981,1.802986e+06,0.0
1867,1.370776e+02,102.424637,2.254750e+03,69.766654,1.075899e+03,72.364154,2.549181e+03,74.444746,54.164354,2.288175e+03,...,1.809806e+04,199.592210,8.916533e+02,74.150909,1.503510e+04,591.901356,278.655945,112.053317,5.753469e+05,0.0
1868,1.931054e+03,1420.996569,2.982484e+04,1019.787832,1.426301e+04,1026.520885,3.374774e+04,1043.562458,772.674959,3.017783e+04,...,2.345420e+05,2795.534148,1.214879e+04,1052.875766,1.988760e+05,7442.279644,3736.488200,1533.728262,8.950014e+06,0.0
1869,2.222956e+03,1636.605709,3.471759e+04,1169.855766,1.659674e+04,1178.951616,3.926862e+04,1201.681742,887.561982,3.508485e+04,...,2.730942e+05,3218.518276,1.406016e+04,1209.554269,2.315408e+05,8627.453161,4332.951420,1774.768085,1.009709e+07,0.0


In [11]:
sum_output = list(mmm2.sum(axis=1))

In [6]:
with open("total_output_sector.json",'r') as load_f:
    total_output_sector = json.load(load_f)

In [8]:
total_output_sector['SAU']['ind']

452445089.9237076

In [43]:
total_output = []
for i in Countries[:-1]:
    for j in A:
        total_output.append(total_output_sector[i][j])
total_output.append(total_output_sector['ROW'])

In [44]:
diff = []
for i in range(len(total_output)):
    diff.append(total_output[i] - sum_output[i])

In [45]:
np.sum(diff),np.max(diff),np.min(diff)

(-0.00158719442602262, 2.574920654296875e-05, -4.57763671875e-05)